Voor installatie gebruik Python 3.9.x . En PyTorch `pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116`
Voor pip packages installeer:
- `pip install transformers`

Goeie tutorial van hoe je de Hugging Face transformers library gebruikt: https://neptune.ai/blog/hugging-face-pre-trained-models-find-the-best

In [1]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import numpy as np
import torch
import tensorflow as tf

c:\Users\ManuV\Documents\Bachelorproef\BP_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = []
tokenizers = []
namen = []

In [3]:
# https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap
# Needs input in format like: "answer: Mona Lisa context: The Mona Lisa is painted by Leonardo DaVinci."
token_t5_squad = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

t5_squad = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

namen.append('T5 model trained on SQuAD v1.1')
models.append(t5_squad)
tokenizers.append(token_t5_squad)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [4]:
# model van https://huggingface.co/wiselinjayajos/t5-end2end-questions-generation

token_t5_modsquad = AutoTokenizer.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")# model begrijpt geen strings dus tokenizer zet de input_text en output_text om naar nummers

t5_modsquad = AutoModelForSeq2SeqLM.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")# pretrained model

namen.append('T5 model trained on modified SQuAD')
models.append(t5_modsquad)
tokenizers.append(token_t5_modsquad)

In [5]:
# https://huggingface.co/iarfmoose/t5-base-question-generator

token_t5_diff = AutoTokenizer.from_pretrained("iarfmoose/t5-base-question-generator")

t5_diff = AutoModelForSeq2SeqLM.from_pretrained("iarfmoose/t5-base-question-generator")

namen.append('T5 model trained on SQuAD, CoQA and MSMARCO')
models.append(t5_diff)
tokenizers.append(token_t5_diff)

# Heeft een QA BERT evaluater ook en kan volledige vragen en MCGs genereren. Kan ook opgeven hoeveel vragen moet genereren. 

In [6]:
token_bart_squad = AutoTokenizer.from_pretrained("p208p2002/bart-squad-qg-hl")

bart_squad = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/bart-squad-qg-hl")

namen.append('BART model trained on SQuAD')
models.append(bart_squad)
tokenizers.append(token_bart_squad)

In [7]:
token_bart_unknown = AutoTokenizer.from_pretrained("kaejo98/bart-base-question-generation")

bart_unknown = AutoModelForSeq2SeqLM.from_pretrained("kaejo98/bart-base-question-generation")

namen.append('BART model trained on unknown dataset')
models.append(bart_unknown)
tokenizers.append(token_bart_unknown)

In [8]:
token_bart_eqg = AutoTokenizer.from_pretrained("voidful/bart-eqg-question-generator")

bart_eqg = AutoModelForSeq2SeqLM.from_pretrained("voidful/bart-eqg-question-generator")

namen.append('BART model trained on RACE-EQG')
models.append(bart_eqg)
tokenizers.append(token_bart_eqg)

In [9]:
token_bart_backtrain = AutoTokenizer.from_pretrained("McGill-NLP/bart-qg-mlquestions-backtraining")

bart_backtrain = AutoModelForSeq2SeqLM.from_pretrained("McGill-NLP/bart-qg-mlquestions-backtraining")

namen.append('BART model unsupervised trained(MLQuestions) with a method called backtraining')
models.append(bart_backtrain)
tokenizers.append(token_bart_backtrain)

In [10]:
text = "Napoleon Bonaparte (born Napoleone Buonaparte; 15 August 1769 - 5 May 1821), later known by his regnal name Napoleon I, was a French military commander and political leader who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804, then Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy endures to this day, as a highly celebrated and controversial leader. He initiated many liberal reforms that have persisted in society, and is considered one of the greatest military commanders in history. His wars and campaigns are studied by militaries all over the world. Between three and six million civilians and soldiers perished in what became known as the Napoleonic Wars."
text_hl = "Napoleon Bonaparte (born Napoleone Buonaparte; 15 August 1769 - 5 May 1821), later known by his regnal name Napoleon I, was a French military commander and political leader who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804, then Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy endures to this day, as a highly celebrated and controversial leader. He initiated many liberal reforms that have persisted in society, and is considered one of the greatest military commanders in history. His wars and campaigns are studied by militaries all over the world. [HL]Between three and six million civilians and soldiers[HL] perished in what became known as the Napoleonic Wars."
text_answer_context = f"answer: Between three and six million civilians and soldiers context: {text} </s>"

for naam,model,tokenizer in zip(namen,models,tokenizers):
    print(naam)
    if naam == 'BART model trained on SQuAD':
        text = text_hl
    elif naam == 'T5 model trained on SQuAD v1.1':
        text = text_answer_context
    tokenized_text = tokenizer([text])
    input_ids = torch.tensor(tokenized_text["input_ids"], dtype=torch.long)
    generated_text = model.generate(input_ids=input_ids,max_length=20)[0]
    result_list = tokenizer.batch_decode(generated_text)
    result = ' '.join(result_list)
    print('\n'+result+'\n')
    

T5 model trained on SQuAD v1.1

<pad> question : How many people died in Napoleon ic Wars ? </s>

T5 model trained on modified SQuAD

<pad> Napoleon Bon a part e was  a French military commander and political leader who rose to prominence during

T5 model trained on SQuAD, CoQA and MSMARCO

<pad> Between three and six million civilian s and soldiers per ished in what became known as the Napoleon ic

BART model trained on SQuAD

</s> How  many  people  died  in  the  Nap ole onic  Wars ? </s>

BART model trained on unknown dataset

</s> <s> What  was  Napoleon  Bon ap arte 's  reg n al  name ? </s>

BART model trained on RACE-EQG

</s> how  many  people  lost  their  lives  in  the  Nap ole onic  Wars ? </s>

BART model unsupervised trained(MLQuestions) with a method called backtraining

</s> What  is  the  role  of  Bon ap arte  in  the  French  Revolution </s>

